In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from autots import AutoTS
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing,ARIMA
from scipy.stats import pearsonr

In [ ]:
#### prepare data
data_raw = pd.read_csv("/home/ylian/SIMDIR/dataset_mas/1/train",sep="|")
data_raw.columns = ["Date","bbid","sales","nan"]
data_raw = data_raw.drop(["nan"],axis=1)
data_clean = data_raw.dropna()

In [ ]:
#### module
bbid_choose = "EQ0000000010970637"
df = data_clean[data_clean["bbid"] == bbid_choose]
df = df.drop(["bbid"],axis=1)
bbid_pick1_date = df["Date"]
bbid_pick1_date = [ datetime.strptime(str(x), '%Y%m%d').strftime('%Y-%m') for x in df["Date"] ]
df["Month"] = bbid_pick1_date
df = df.drop(["Date"],axis=1)
df = df.groupby("Month").sum()
df.to_csv("temp")
data = pd.read_csv('/home/ylian/SIMDIR/temp')
series = TimeSeries.from_dataframe(data, 'Month', 'sales')# Splitting the series in train and validation set
train, val = series.split_before(pd.Timestamp('20200101'))
model = ARIMA()
model.fit(train)
prediction = model.predict(len(val))
series.plot()
prediction.plot(label='forecast', low_quantile=0.05, high_quantile=0.95)
plt.title("CHGG sales prediction")
plt.legend()